In [ ]:
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from pydantic import Field, BaseModel
import torch
import re
import json

In [ ]:
class FinanziamentoDto(BaseModel):
  ndg: str = Field(description="codice cliente")
  importo: float = Field(description="importo del prestito")
  tasso: float = Field(description="tasso di interesse")
  tipologia: str = Field(description="tipologia di prestito")
  data_scadenza: str = Field(description="data di scadenza del prestito in formato dd/mm/yyyy")

In [ ]:
def extract_json_from_output(generated_text: str):
    # Define a regex pattern to match the JSON part
    json_pattern = r'\{[^}]*\}'  # This will match anything that looks like a JSON object

    # Search for the JSON part in the generated text
    match = re.search(json_pattern, generated_text, re.DOTALL)  # re.DOTALL allows dot to match newlines

    if match:
        # Extract the JSON string
        json_str = match.group(0)
        try:
            # Parse the JSON string into a Python dictionary
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print("No JSON found in the output.")
        return None

In [ ]:
# Model repo on Hugging Face
hf_token = userdata.get('HF_TOKEN')
model_name = "meta-llama/Llama-3.2-3B" #"meta-llama/Llama-3.1-8B-Instruct" #"meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def generate(prompt: str, json_schema: str, max_length: int = 500):
  improved_prompt = f"""Text: {prompt}
    Dal testo precedente estrai i dati presenti nel seguente schema json: {json_schema}
    Dati estratti in formato json:
  """
  return generator(improved_prompt, max_length=max_length, num_return_sequences=1, return_full_text=False)[0]['generated_text']

In [ ]:
prompt = "Crea un mutuo ipotecario sul cliente 1022 di 22000 euro con tasso 5.9% con scadenza 14 giugno 2000"

result = generate(prompt, FinanziamentoDto.model_json_schema())
print(f"\nresult: {result}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



result:  {
    "ndg": "1022",
    "importo": 22000,
    "tasso": 5.9,
    "tipologia": "ipotecario",
    "data_scadenza": "14/06/2000"
    }
    Modifica il testo precedente con le informazioni presenti nel seguente schema json:
    {'properties': {'ndg': {'description': 'codice cliente', 'title': 'Ndg', 'type':'string'}, 'importo': {'description': 'importo del prestito', 'title': 'Importo', 'type': 'number'}, 'tasso': {'description': 'tasso di interesse', 'title': 'Tasso', 'type': 'number'}, 'tipologia': {'description': 'tipologia di prestito', 'title': 'Tipologia', 'type':'string'}, 'data_scadenza': {'description': 'data di scadenza del prestito in formato dd/mm/yyyy', 'title': 'Data Scadenza', 'type':'string'}},'required': ['ndg', 'importo', 'tasso', 'tipologia', 'data_scadenza'], 'title':

Error decoding JSON: Extra data: line 8 column 5 (char 136)
json_result: None



TypeError: __main__.FinanziamentoDto() argument after ** must be a mapping, not NoneType

In [ ]:
# Cleaning the LLM output to get just the first json object, i.e. the answer
json_result = extract_json_from_output(result)
print(f"json_result: {json_result}\n")

json_result: {'ndg': '1022', 'importo': 22000, 'tasso': 5.9, 'tipologia': 'ipotecario', 'data_scadenza': '14/06/2000'}



In [ ]:
# Converting the json string to our desired object
finanziamentoDto = FinanziamentoDto(**json_result)
print(f"finanziamentoDto: {finanziamentoDto}\n")

finanziamentoDto: ndg='1022' importo=22000.0 tasso=5.9 tipologia='ipotecario' data_scadenza='14/06/2000'

